In [1]:
from snowflake.core import Root
from snowflake.snowpark import Session
from snowflake.snowpark.functions import udtf, col
from snowflake.snowpark.types import StringType, IntegerType, StructType, StructField
from snowflake.cortex import Complete

import langchain

import json

import logging
logging.basicConfig(level=logging.ERROR)

In [2]:
connection_parameters = json.load(open('connection.json'))
session = Session.builder.configs(connection_parameters).create()
session.sql_simplifier_enabled = True

In [3]:
# create database
session.sql('create database if not exists sec_cortex_demo').collect()
session.sql('use schema sec_cortex_demo.public').collect()

[Row(status='Statement executed successfully.')]

### Test Complete with foundation models

In [4]:
Complete('mistral-7b',
         'How many customers does Snowflake have?')

" I don't have real-time access to Snowflake's internal data, but I can provide you with some publicly available information. According to Snowflake's Q3 2022 earnings report, they had approximately 5,300 active customers as of October 31, 2022. This number includes both paid and free trial customers. Please note that this figure may not reflect the exact number of unique organizations using Snowflake, as some organizations may have multiple accounts or departments using the platform. For the most accurate and up-to-date information, I would recommend checking Snowflake's investor relations website or contacting their investor relations team directly."

In [5]:
Complete('mistral-large',
         'How many customers does Snowflake have?')

" I don't have real-time data access to provide the current number of customers Snowflake has. As of a certain point in time, Snowflake, the cloud-based data warehousing platform, had several thousand customers worldwide. However, for the most accurate and up-to-date information, I would recommend checking Snowflake's official website or their latest press releases."

### Prepare dataset for RAG
1. Get [Cybersyn LLM Training Essentials](https://app.snowflake.com/marketplace/listing/GZTSZ290BUX1X/cybersyn-llm-training-essentials) dataset from Snowflake Marketplace.
2. Create dataset for all companies with SIC code 7323.
3. Create UDTF for chunking data.
4. Chunk and embed data using [SNOWFLAKE.CORTEX.EMBED_TEXT_768](https://docs.snowflake.com/en/sql-reference/functions/embed_text-snowflake-cortex).

In [6]:
# join tables from Cybersyn LLM Training Essentials dataset. 
session.sql(
"""
create or replace table sec_reports_base as 
SELECT
    txt.sec_document_id,
    companies.cik,
    txt.variable_name as document_type,
    companies.company_name,
    companies.sic_code_category,
    companies.sic_code_description,
    companies.country,
    txt.period_end_date,
    txt.value
FROM 
    llm_training_essentials.cybersyn.sec_report_text_attributes AS txt
    JOIN llm_training_essentials.cybersyn.sec_cik_index AS companies ON (companies.cik = txt.cik)
WHERE 
    txt.period_end_date >= '2020-01-01'
    and document_type = '10-K Filing Text'
    and companies.sic = 7372
qualify 
    row_number() over (partition by companies.cik order by period_end_date desc) = 1; 
"""
).collect()

[Row(status='Table SEC_REPORTS_BASE successfully created.')]

In [7]:
session.table("SEC_REPORTS_BASE").limit(10).to_pandas()

,SEC_DOCUMENT_ID,CIK,DOCUMENT_TYPE,COMPANY_NAME,SIC_CODE_CATEGORY,SIC_CODE_DESCRIPTION,COUNTRY,PERIOD_END_DATE,VALUE
0,0001866633-24-000004_10-K,0001866633,10-K Filing Text,"CONSENSUS CLOUD SOLUTIONS, INC.",Office of Technology,SERVICES-PREPACKAGED SOFTWARE,US,2023-12-31,\n \n \n ccs-20231231 UNITED STATES SECURITIES...
1,0001845022-24-000020_10-K,0001845022,10-K Filing Text,"COUCHBASE, INC.",Office of Technology,SERVICES-PREPACKAGED SOFTWARE,US,2024-01-31,\n \n \n \n \n \n base-20240131 Table of Conte...
2,0001213900-23-081208_10-K,0001807166,10-K Filing Text,AMESITE INC.,Office of Technology,SERVICES-PREPACKAGED SOFTWARE,US,2023-06-30,\n \n \n \n \n \n \n \n \n \n \n \n \n U...
3,0001107843-24-000008_10-K,0001107843,10-K Filing Text,"QUALYS, INC.",Office of Technology,SERVICES-PREPACKAGED SOFTWARE,US,2023-12-31,\n \n \n qlys-20231231 Table of Contents UNIT...
4,0001628280-24-007825_10-K,0001866368,10-K Filing Text,"CLEARWATER ANALYTICS HOLDINGS, INC.",Office of Technology,SERVICES-PREPACKAGED SOFTWARE,US,2023-12-31,\n \n \n cwan-20231231 Table of Contents UNITE...
5,0000950170-23-066654_10-K,0001768267,10-K Filing Text,CERENCE INC.,Office of Technology,SERVICES-PREPACKAGED SOFTWARE,US,2023-09-30,\n \n \n \n 10-K \n \n \n \n \n \n \n UNIT...
6,0001437749-24-008626_10-K,0000803578,10-K Filing Text,"WAVEDANCER, INC.",Office of Technology,SERVICES-PREPACKAGED SOFTWARE,US,2023-12-31,\n \n \n \n wavd20231231_10k.htm \n \n \n \n \...
7,0001459200-24-000008_10-K,0001459200,10-K Filing Text,"ALARM.COM HOLDINGS, INC.",Office of Technology,SERVICES-PREPACKAGED SOFTWARE,US,2023-12-31,\n \n \n alrm-20231231 UNITED STATES SECURITIE...
8,0001689923-24-000006_10-K,0001689923,10-K Filing Text,"ALTERYX, INC.",Office of Technology,SERVICES-PREPACKAGED SOFTWARE,US,2023-12-31,\n \n \n ayx-20231231 UNITED STATES SECURITIES...
9,0001773383-23-000055_10-K,0001773383,10-K Filing Text,"DYNATRACE, INC.",Office of Technology,SERVICES-PREPACKAGED SOFTWARE,US,2023-03-31,\n \n dt-20230331 UNITED STATES SECURITIES AND...


In [8]:
# Create chunking UDTF
# Experiment with chunk size and overlap
session.sql('create stage if not exists udtf_stage').collect()
@udtf(name="CHUNK_TEXT", 
      is_permanent=True, 
      packages=["langchain","pandas"], 
      stage_location = '@udtf_stage',
      input_types=[StringType()],
      output_schema=StructType([StructField('chunk', StringType()), StructField('start_index', IntegerType())]),
      replace=True)
class text_chunker:
    def process(self,text): 
        from langchain.text_splitter import RecursiveCharacterTextSplitter
        import pandas as pd       
        text_raw=[]
        text_raw.append(text) 
        
        text_splitter = RecursiveCharacterTextSplitter(
            separators = ["\\n", ". " ], # Define an appropriate separator. New line is good typically!
            chunk_size = 1000, #Adjust this as you see fit
            chunk_overlap  = 200, #This lets text have some form of overlap. Useful for keeping chunks contextual
            length_function = len,
            keep_separator = False,
            add_start_index = True #Optional but useful if you'd like to feed the chunk before/after
        )
    
        chunks = text_splitter.create_documents(text_raw)
        df = pd.DataFrame(chunks, columns=['chunks','meta'])

        df['meta'] = df['meta'].apply(lambda x: x[1]['start_index']).astype(int)
        df['chunks'] = df['chunks'].apply(lambda x: x[1])
        
        yield from df.itertuples(index=False, name=None)

In [9]:
# chunk dataset and embed content using snowflake.cortex.embed_text_768
session.sql(
"""
create or replace table content_chunks_10k as 
select 
    sec_document_id,
    document_type,
    company_name,
    sic_code_category,
    sic_code_description,
    country,
    period_end_date,
    chunk as content_chunk,
    snowflake.cortex.embed_text_768('e5-base-v2', content_chunk) embedding,
    start_index,
    row_number() over (partition by sec_document_id order by sec_document_id, start_index) as document_index_rownum,
    row_number() over (order by sec_document_id, start_index) as rownum
from 
    sec_reports_base,
    table(chunk_text(value))
""").collect()

[Row(status='Table CONTENT_CHUNKS_10K successfully created.')]

#### Examine the chunks

In [10]:
(session.table("content_chunks_10k")
                 .filter(col("COMPANY_NAME") == "SNOWFLAKE INC.")
                 .order_by(col("ROWNUM"))
                 .limit(1)
                 .collect()
        )

[Row(SEC_DOCUMENT_ID='0001640147-24-000101_10-K', DOCUMENT_TYPE='10-K Filing Text', COMPANY_NAME='SNOWFLAKE INC.', SIC_CODE_CATEGORY='Office of Technology', SIC_CODE_DESCRIPTION='SERVICES-PREPACKAGED SOFTWARE', COUNTRY='US', PERIOD_END_DATE=datetime.date(2024, 1, 31), CONTENT_CHUNK='snow-20240131 Table  of Contents  UNITED STATES SECURITIES AND EXCHANGE COMMISSION Washington, D.C. 20549 FORM  10-K   (Mark One) ☒ ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934  For the fiscal year ended  January 31 , 2024 OR ☐ TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934 For the transition period from           to   Commission file number:  001-39504   SNOWFLAKE INC.   (Exact name of registrant as specified in its charter) Delaware 46-0636374 (State or other jurisdiction of incorporation or organization) (I.R.S', EMBEDDING=[-0.022220227867364883, -0.020755713805556297, -0.04289441183209419, -0.022506648674607277, 0.04646274

In [11]:
chunks = (session.table("content_chunks_10k")
                 .filter(col("COMPANY_NAME") == "SNOWFLAKE INC.")
                 .order_by(col("ROWNUM"))
                 .collect()
        )

# these indexes will be different if you use different chunk or overlap settings
print(f"Total chunks {len(chunks)}\n")

print("End of chunk 1:")
print(chunks[0]["CONTENT_CHUNK"][457:])
print("\n")

print("Beginning of chunk 2:")
print(chunks[1]["CONTENT_CHUNK"][:200])

Total chunks 574

End of chunk 1:
(Exact name of registrant as specified in its charter) Delaware 46-0636374 (State or other jurisdiction of incorporation or organization) (I.R.S


Beginning of chunk 2:
(Exact name of registrant as specified in its charter) Delaware 46-0636374 (State or other jurisdiction of incorporation or organization) (I.R.S. Employer Identification No.) Suite 3A ,  106 East Babc


### Use Vector Distance Functions to find information relevant to users questions

Retrieve relevant information from the database

In [12]:
user_question = "How many customers does Snowflake have?"
res = session.sql(
f"""
select 
    content_chunk
from 
    content_chunks_10k
where 
    company_name = 'SNOWFLAKE INC.'
order by 
    vector_cosine_distance(embedding, 
                           snowflake.cortex.embed_text_768('e5_base_v2', '{user_question}')
                           ) desc
limit 3                
""").collect()

for r in res:
    print(r["CONTENT_CHUNK"])

See the section titled “Key Business Metrics” for a definition of net revenue retention rate. Our platform is used globally by organizations of all sizes across a broad range of industries. As of January 31, 2024, we had 9,437 total customers, increasing from 7,744 customers as of January 31, 2023. Our customer count is subject to adjustments for acquisitions, consolidations, spin-offs, and other market activity, and we present our total customer count for historical periods reflecting these adjustments. Our platform has been adopted by many of the world’s largest organizations that view Snowflake as a key strategic partner in their cloud and data transformation initiatives. As of January 31, 2024, our customers included 691 of the Forbes Global 2000, based on the 2023 Forbes Global 2000 list, and those customers contributed approximately 41% of our revenue for the fiscal year ended January 31, 2024
Our platform is the innovative technology that powers the Data Cloud, enabling customer

Feed retreived information from database into an LLM prompt and use `Complete` fucntion.

In [13]:
user_question = "How many customers does Snowflake have?"
session.sql(f"""
select 
    snowflake.cortex.complete('mistral-7b',
                              concat_ws(chr(10), 
                                        'Answer the question using the context provided.',
                                        'Be concise.',
                                        'Context:',
                                        '####',
                                        content_chunk,
                                        '####',
                                        'Question:',
                                        '{user_question}')
                            ) as llm_response
from 
    content_chunks_10k
where 
    company_name = 'SNOWFLAKE INC.'
order by 
    vector_cosine_distance(embedding, 
                           snowflake.cortex.embed_text_768('e5_base_v2', '{user_question}')
                           ) desc
limit 1               
""").collect()[0]["LLM_RESPONSE"]

' As of January 31, 2024, Snowflake had 9,437 customers.'

### Use Cortex Search Service
Cortex Search does not use the `embed_text_868` function output when we create `content_chunks_10k` above. The Cortex Search Service handles our embeddings for us. 

[https://docs.snowflake.com/LIMITEDACCESS/cortex-search/query-cortex-search-service](https://docs.snowflake.com/LIMITEDACCESS/cortex-search/query-cortex-search-service)

In [14]:
# create search service on chunked data
session.sql(
"""
CREATE OR REPLACE CORTEX SEARCH SERVICE SEC_10K_SEARCH_SERVICE
  ON content_chunk
  ATTRIBUTES company_name
  WAREHOUSE = xsmall
  TARGET_LAG = '1 hour'
  AS (
    SELECT
        content_chunk,
        company_name,
        sec_document_id,
        start_index
    FROM content_chunks_10k
    )
""").collect();

In [15]:
# access search service through Python API
root = Root(session)
search_service = (root
                  .databases["SEC_CORTEX_DEMO"]
                  .schemas["PUBLIC"]
                  .cortex_search_services["SEC_10K_SEARCH_SERVICE"]
)

In [16]:
user_question = "How many customers does Snowflake have?"
resp = search_service.search(
    user_question,
    columns=["CONTENT_CHUNK","SEC_DOCUMENT_ID"],
    filter={"@eq": {"COMPANY_NAME": "SNOWFLAKE INC."} },
    limit=3,
)
[c["CONTENT_CHUNK"] for c in resp.results]

['See the section titled “Key Business Metrics” for a definition of net revenue retention rate. Our platform is used globally by organizations of all sizes across a broad range of industries. As of January 31, 2024, we had 9,437 total customers, increasing from 7,744 customers as of January 31, 2023. Our customer count is subject to adjustments for acquisitions, consolidations, spin-offs, and other market activity, and we present our total customer count for historical periods reflecting these adjustments. Our platform has been adopted by many of the world’s largest organizations that view Snowflake as a key strategic partner in their cloud and data transformation initiatives. As of January 31, 2024, our customers included 691 of the Forbes Global 2000, based on the 2023 Forbes Global 2000 list, and those customers contributed approximately 41% of our revenue for the fiscal year ended January 31, 2024',
 '79 Table of Contents SNOWFLAKE INC. NOTES TO CONSOLIDATED FINANCIAL STATEMENTS 1.

Create prompt

In [17]:
context = '\n'.join([c["CONTENT_CHUNK"] for c in resp.results])

prompt = f"""
Use the context provided to answer the question. Be concise. 
Context:
####
{context}
####
Question: {user_question}
"""
print(prompt)


Use the context provided to answer the question. Be concise. 
Context:
####
See the section titled “Key Business Metrics” for a definition of net revenue retention rate. Our platform is used globally by organizations of all sizes across a broad range of industries. As of January 31, 2024, we had 9,437 total customers, increasing from 7,744 customers as of January 31, 2023. Our customer count is subject to adjustments for acquisitions, consolidations, spin-offs, and other market activity, and we present our total customer count for historical periods reflecting these adjustments. Our platform has been adopted by many of the world’s largest organizations that view Snowflake as a key strategic partner in their cloud and data transformation initiatives. As of January 31, 2024, our customers included 691 of the Forbes Global 2000, based on the 2023 Forbes Global 2000 list, and those customers contributed approximately 41% of our revenue for the fiscal year ended January 31, 2024
79 Table o

Feed prompt to LLM to get a user-friendly response

In [18]:
Complete('mistral-7b',
         prompt
)

' As of January 31, 2024, Snowflake had 9,437 total customers.'